<a href="https://colab.research.google.com/github/vargheesk/IMDB-Sentimental-analysis-using-RNN/blob/main/IMDB_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
import numpy as np
import re

In [2]:
df=pd.read_csv("/content/IMDB Dataset.csv")

In [3]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
sw=stopwords.words("english")


In [6]:
from nltk.chunk.regexp import regex
def preprocessing(df):
  x_data=df["review"]
  y_data=df["sentiment"]
  x_data =x_data.replace({"<.*?>" :"","[^A-Za-z ]":""},regex=True)
  x_data= x_data.apply(lambda review :[word.lower() for word in review.split() if word.lower() not in sw])
  y_data=y_data.replace({"negative" :0,"positive":1})
  # print(y_data)
  return x_data,y_data

In [7]:
x,y = preprocessing(df)

/tmp/ipython-input-3889897368.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data=y_data.replace({"negative" :0,"positive":1})


In [8]:
x

,review
0,"[one, reviewers, mentioned, watching, oz, epis..."
1,"[wonderful, little, production, filming, techn..."
2,"[thought, wonderful, way, spend, time, hot, su..."
3,"[basically, theres, family, little, boy, jake,..."
4,"[petter, matteis, love, time, money, visually,..."
...,...
49995,"[thought, movie, right, good, job, wasnt, crea..."
49996,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,"[catholic, taught, parochial, elementary, scho..."
49998,"[im, going, disagree, previous, comment, side,..."


In [22]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1,test_size=.2)

In [10]:
rev_length =[]
for rev in x_train:
  rev_length.append(len(rev))
rev_length.sort()
max_length = rev_length[-1]

In [11]:
avg_length=int(np.mean(rev_length))
print(avg_length)

118


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
# tokenizer.word_index

In [13]:
x_train1=tokenizer.texts_to_sequences(x_train)
x_test1=tokenizer.texts_to_sequences(x_test)

In [14]:
x_train_pad=pad_sequences(x_train1,maxlen=max_length,padding="post",truncating="post")
x_test_pad=pad_sequences(x_test1,maxlen=max_length,padding="post",truncating="post")

In [15]:
voc_size=len(tokenizer.word_index)+1
print(voc_size)

187060


In [16]:
model=Sequential()
model.add(Embedding(voc_size,32,input_length=avg_length))
model.add(LSTM(12))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
checkpoint=ModelCheckpoint("model/LSTM.h5",monitor="accuracy",save_best_only=True,mode="auto")

In [18]:
model.fit(x_train_pad,y_train,epochs=20,batch_size=64,callbacks=[checkpoint])


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5019 - loss: 0.6933

625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 42ms/step - accuracy: 0.5018 - loss: 0.6933
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.4956 - loss: 0.6933
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4966 - loss: 0.6932

625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.4966 - loss: 0.6932
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5010 - loss: 0.6932

625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.5010 - loss: 0.6932
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.5010 - loss: 0.6932
Epoch 6/20
624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4997 - loss: 0.6932

625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.4997 - loss: 0.6932
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.5034 - loss: 0.6931
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.5006 - loss: 0.6932
Epoch 9/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - accuracy: 0.5016 - loss: 0.6932
Epoch 10/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.5034 - loss: 0.6932
Epoch 11/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.5017 - loss: 0.6932
Epoch 12/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.4988 - loss: 0.6932
Epoch 13/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.4956 - loss: 0.6932
Epoch 14/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.4974 - loss: 0.6932
Epoch 15/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.4999 - loss: 0.6932
Epoch 16/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.4989 - loss: 0.6932
Epoch 17/20
625/625 ━━━━━━━━━━━━━━━━

In [19]:
model=load_model("/content/model/LSTM.h5")

In [27]:
y_pred=model.predict(x_test_pad)

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step


In [28]:
y_pred1=y_pred>0.5
y_pred1

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5044
           1       0.50      1.00      0.66      4956

    accuracy                           0.50     10000
   macro avg       0.25      0.50      0.33     10000
weighted avg       0.25      0.50      0.33     10000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
def predict(review):
  review=re.sub('[^a-zA-Z ]','',review)
  # print(review)
  filtered=[word.lower()for word in review.split() if word.lower() not in sw]
  # print(filtered)
  filtered=' '.join(filtered)
  # print(filtered)
  tokenizer_word=tokenizer.texts_to_sequences([filtered])
  # print(tokenizer_word)
  padded_word=pad_sequences(tokenizer_word,maxlen=max_length,padding="post",truncating="post")
  # print(padded_word)
  prediction=model.predict(padded_word)
  print("positive" if prediction >0.5 else "negative")

In [31]:
predict_new=predict("it is a not good film!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
positive


In [ ]:
# from typing import Sequence
# Sequence=tokenizer.texts_to_sequences(df["review"])

In [32]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [35]:
!git config --global user.email "vargheeskutty007@gmail.com"

In [36]:
!git config --global user.name "vargheesk"

In [37]:

!git commit -m "first commit"
!git branch -M main


On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	IMDB Dataset.csv
	model/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
error: remote origin already exists.
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/vargheesk/IMDB-Sentimental-analysis-using-RNN.git'


In [38]:
!git remote add origin https://github.com/vargheesk/IMDB-Sentimental-analysis-using-RNN.git
!git push -u origin main

error: remote origin already exists.
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/vargheesk/IMDB-Sentimental-analysis-using-RNN.git'


In [39]:
!docker images

/bin/bash: line 1: docker: command not found


In [40]:
!docker run hello-world

/bin/bash: line 1: docker: command not found


In [42]:
!wsl --set-default-version 2

/bin/bash: line 1: wsl: command not found
